In [1]:
! pip install transformers
# ! pip install transformers<4.38
# ! pip install tensorflow==2.11.0 keras==2.11.0

In [2]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# # Raw URL
# normal = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/nomal_data.csv"
# train =  "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/train.csv"
# test = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/test.csv"
# instagram = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/instagram.csv"


# # CSV 파일 읽기
# normal_data = pd.read_csv(normal)
# train_data = pd.read_csv(train)
# test_data = pd.read_csv(test)

# #  normal 데이터 형식 맞추기
# normal = normal_data
# normal.reset_index(inplace=True)
# normal['class'] = '일반 대화'
# normal.columns = ['idx','conversation', 'class']                                                 # 칼럼명 변경
# normal = normal[['idx','class', 'conversation']]

# # normal과 train 합치기
# train_data = pd.concat([train_data, normal_data[:1000]], ignore_index=True)                         # nomal_data에서 1000개만 뽑아서 합치기
# train_data = train_data.drop('idx', axis=1).sample(frac=1).reset_index(drop=True)               # index 삭제하고 데이터행 섞기
# train_data.reset_index(inplace=True)                                                            # index 다시 붙이기
# train_data.columns = ['idx','class', 'conversation']                                            # 칼럼명 변경

# train_data                  # 일반 대화 잘 섞였는지 확인

In [4]:
# Raw URL
# normal = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/nomal_data.csv"
train =  "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/train.csv"
test = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/test.csv"
instagram = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/instagram.csv"


# CSV 파일 읽기
# normal_data = pd.read_csv(normal)
train_data = pd.read_csv(train)
test_data = pd.read_csv(test)
instagram_data = pd.read_csv(instagram)

train_data = pd.concat([train_data, instagram_data], ignore_index=True)  
train_data = train_data.drop('idx', axis=1).sample(frac=1).reset_index(drop=True)               # index 삭제하고 데이터행 섞기
train_data.reset_index(inplace=True)                                                            # index 다시 붙이기
train_data.columns = ['idx','class', 'conversation']                                            # 칼럼명 변경

train_data.head()                  # 일반 대화 잘 섞였는지 확인

,idx,class,conversation
0,0,협박 대화,너 돈 언제 갚을꺼야?\n미안해. 다음달까지 기달려줘\n벌써 6개월이나 더 기달려줬...
1,1,기타 괴롭힘 대화,안녕하세요 복자식당입니다.\n 야 장난하냐?\n네?\n아니 배달시킨지 40분이 지나...
2,2,기타 괴롭힘 대화,너 이거 할 줄 알아 ?\n당연하지 ! 나 할 수 있어 !\n진짜로 ??? 믿음이 ...
3,3,협박 대화,자꾸 이렇게 나오면 나도 가만 안있을거야!\n매번 이런식으로 나오시는데 저희도 할만...
4,4,협박 대화,어쩔 수 없다. 네가 죽어줘야겠다.\n살려주십쇼 형님!\n넌 너무 많은 걸 알고 있...


In [5]:
# label_encoder = LabelEncoder()
# label_encoder.fit(train_data['class'])
# num_labels = len(label_encoder.classes_)

# train_data['encoded_label'] = np.asarray(label_encoder.transform(train_data['class']), dtype=np.int32)
# train_data.head()

In [6]:
train_texts = train_data["conversation"].to_list() # Features (not-tokenized yet)
# train_labels = train_data["encoded_label"].to_list() # Labels
train_class = train_data["class"].to_list() # Labels

In [7]:
# validation data 분리

from sklearn.model_selection import train_test_split

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_class, test_size=0.2, random_state=0)

### 토크나이저 다운로드

In [8]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='<s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# 시작 토큰, 종료 토큰 붙이기

import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convert_examples_to_features(sent_list, max_seq_len, tokenizer):
    input_ids = []

    for conversation in tqdm(sent_list, total=len(sent_list)):
        bos_token = [tokenizer.bos_token]  # 시작 토큰
        eos_token = [tokenizer.eos_token]  # 종료 토큰
        unused_token = ['<unused0>']  # 마지막에 추가할 토큰

        # '\n'으로 문장들 분리
        sentences = conversation.split("\n")

        tokens = []  # 전체 토큰 리스트

        for i, sentence in enumerate(sentences):
            # 각 문장에 <s>와 </s> 추가
            sentence_tokens = bos_token + tokenizer.tokenize(sentence) + eos_token
            tokens += sentence_tokens  # 문장 뒤에 <s> 추가

        # 마지막 문장 뒤에 <unused0> 추가
        tokens += unused_token  # 마지막에 <unused0> 추가
        tokens += eos_token  # 마지막에 종료 토큰 추가

        # 정수 인코딩
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 패딩 추가
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        # 길이 확인
        assert len(input_id) == max_seq_len, f"Error with input length {len(input_id)} vs {max_seq_len}"

        # 결과 저장
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids


In [10]:
# maxlen 계산하기

total_data_text = list(train_texts) + list(train_class)
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))
 
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

문장길이 평균 :  113.78832772166105
문장길이 최대 :  874
문장길이 표준편차 :  140.98432257809353
pad_sequences maxlen :  395
전체 문장의 0.9542087542087542%가 maxlen 설정값 이내에 포함됩니다. 


In [11]:
max_seq_len = 330

# 학습데이터 전처리
X_train = convert_examples_to_features(train_texts, max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 3960/3960 [00:04<00:00, 959.15it/s] 


In [12]:
# 최대 길이: 128
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [    0  9367  7244  8236  8152 10721  9036 10623  9098     1     0  9050
  9265  7162 10721  8135  9183  9109  7810  7788  8084  9435  7182  7426
 10276 17624  6872 16913  7182     1     0 10721  8135  9183 46989  9217
   406     1     0 42487  9367  7244  8236  8152 19844 20337 36658  9668
  9036 10623 13023  8263     1     0  9265  9990  7312 10924 15859  9183
  9109  7810  7788  8084     1     0 23097  9449  9046  7177  7587  7162
  6853  6889 10721 11459  9180  9018 10721  9046 20108 22051  6853  7991
     1     0  9078  8702  8702 10330 19844  7177 15370 11001 31896  8702
  7895  7172  7182  9903  7469  9036 10623  9098  9367  7244  8236  8152
     1     0 36510     1     0 10165 10721 11459  9023 17831  7791  7609
  7285 30974 12904 10721  7235 34166 13023 17831  7235 25783 13348 13059
  9325 11830   739  8616  8616 12097 26056     1     0 36510     1     9
     1     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     

In [13]:
# 검증데이터 전처리

X_val = convert_examples_to_features(val_texts, max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 990/990 [00:01<00:00, 986.87it/s] 


In [14]:
# 라벨데이터 정수 라벨링

train_label = train_labels
val_label = val_labels

idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

# 고유한 정수로 변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print('각 레이블과 정수 :', label_idx)


각 레이블과 정수 : {'갈취 대화': 0, '기타 괴롭힘 대화': 1, '일상 대화': 2, '직장 내 괴롭힘 대화': 3, '협박 대화': 4}


In [15]:
# GPT 모델 불러오기

model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [16]:
# GPT 모델 input, output 만들기
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [17]:
# GPT로 다중 분류 모델 만들기

class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [18]:
# # TPU 작동을 위한 코드
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [19]:
# 모델 생성
from tensorflow.keras.metrics import Recall

model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=5)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
# recall_metric = Recall()
# model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', recall_metric])
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.6.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'lm_head.weight', 'transformer.h.11.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [20]:
# y_train = tf.squeeze(y_train)  # (batch_size, 1) -> (batch_size,)
# y_val = tf.squeeze(y_val)  # (batch_size, 1) -> (batch_size,)

# print(y_train.shape)
# print(y_val.shape)

In [21]:
model.fit(
    X_train, y_train, epochs=2, batch_size=16, validation_data = (X_val, y_val)
)

Epoch 1/2
248/248 [==============================] - 337s 1s/step - loss: 0.9041 - accuracy: 0.6975 - val_loss: 0.4283 - val_accuracy: 0.8556
Epoch 2/2
248/248 [==============================] - 324s 1s/step - loss: 0.3380 - accuracy: 0.8846 - val_loss: 0.5032 - val_accuracy: 0.8495


In [24]:
# test_data 전처리 (\n으로 구분하지 않고 대화문 통으로 하나의 문장으로 설정)

def preprocess_test_data_without_newline(test_data, tokenizer, max_seq_len=128):
    # test_data에서 conversation만 추출
    test_texts = test_data["text"].to_list()  # Features (not-tokenized yet)

    # 전처리 진행
    def convert_examples_to_features_for_test(sent_list, max_seq_len, tokenizer):
        input_ids = []

        for conversation in tqdm(sent_list, total=len(sent_list)):
            bos_token = [tokenizer.bos_token]  # 시작 토큰
            eos_token = [tokenizer.eos_token]  # 종료 토큰
            unused_token = ['<unused0>']  # 마지막에 추가할 토큰

            # conversation을 하나의 문장으로 간주
            sentence = conversation.strip()

            tokens = bos_token + tokenizer.tokenize(sentence) + eos_token  # 토큰화 후 <s>, </s> 추가

            # 마지막에 <unused0> 추가
            tokens += unused_token  # 마지막에 <unused0> 추가
            tokens += eos_token  # 마지막에 종료 토큰 추가

            # 정수 인코딩
            input_id = tokenizer.convert_tokens_to_ids(tokens)
            # 패딩 추가
            input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

            # 길이 확인
            assert len(input_id) == max_seq_len, f"Error with input length {len(input_id)} vs {max_seq_len}"

            # 결과 저장
            input_ids.append(input_id)

        input_ids = np.array(input_ids, dtype=int)

        return input_ids

    # test_data에 대해 전처리 진행
    X_test = convert_examples_to_features_for_test(test_texts, max_seq_len, tokenizer)

    return X_test

# 테스트 데이터 전처리 실행
X_test = preprocess_test_data_without_newline(test_data, tokenizer, max_seq_len=128)

print("Test data preprocessing 완료!")


100%|██████████| 500/500 [00:00<00:00, 2044.95it/s]

Test data preprocessing 완료!


In [25]:
# 레이블과 정수 매핑 딕셔너리
label_mapping = {0: 1, 1: 3, 2: 4, 3: 2, 4: 0}

# 예측 수행
predictions = model.predict(X_test)

# 예측 결과가 클래스 확률일 경우
predicted_classes = np.argmax(predictions, axis=1)

# 정수 값을 레이블로 변환
predicted_labels = [label_mapping[class_id] for class_id in predicted_classes]

# 예측 결과를 test 데이터프레임에 추가
test_data['predictions'] = predicted_labels
test_data.to_csv("test.csv",index=False, encoding='utf-8-sig')


# 결과 출력
test_data.head()

,idx,text,predictions
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,1
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,2
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,2
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...,3
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,3


In [26]:
test_data.sample(10)

,idx,text,predictions
232,t_232,아 진짜 못참겠네 왜그래 저번부터 그냥 거슬린다 아니 뭘 잘못했는데 맞자 일단 어디...,3
129,t_129,야 내꺼 겁나 맛없어 내거도 그래 여기 개별로다 그니까 여기 리뷰 안좋은 이유가 있...,3
414,t_414,지은씨 이번 주말에 뭐 해요 저 약속은 없어요 그러면 같이 영화볼래요 요즘 재미있는...,2
41,t_041,야 거기 이 골목 지나려면 통행비 5만원이다. 그딴게 어딨어요? 오늘부터다 시발롬아...,1
483,t_483,이거 왜 이렇게 놔! 똑바로 놓으라고 했지? 아침부터 왜 그러냐. 아니 수저통 똑바...,3
172,t_172,이거 재연씨 아이디어인가? 네 그렇습니다. 아니지 아니지. 무슨 말씀이신지? 지금부...,2
389,t_389,담배 한갑줘 손님 죄송하지만 마스크 착용 부탁드립니다 나 없는데 마스크 그럼 ...,1
332,t_332,온가족이 한날한시에 함께 죽으면 외롭지는 않겠네 저희 가족들한테까지 왜 그러십니까....,0
36,t_036,저기요 번호 주세요 네? 죄송합니다. 머라고요? 내가 누군줄 알아요? 아. 네 당연...,1
482,t_482,아가씨 이쁜데 왜 이러세요 우리랑 좀 놀까? 싫어요. 그럼 돈 좀 빌려주던가 얼마나...,1


In [28]:
# submission 넣기

# submission = "https://raw.githubusercontent.com/seoyeokim/Aiffel_DLThon/main/data/submission.csv"

# submission_data = pd.read_csv(submission)


submission = pd.DataFrame({"idx" : test_data.idx, "class" : predicted_labels})
submission.to_csv("submission.csv",index=False, encoding='utf-8-sig')

submission.head()

,idx,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,3
4,t_004,3
